# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Load and process the data

In [24]:
data_path_1 = "data/prices_round_3_day_0.csv"
data_path_2 = "data/prices_round_3_day_1.csv"
data_path_3 = "data/prices_round_3_day_2.csv"

data_paths = [data_path_1, data_path_2, data_path_3]

# read the data and concatenate them
all_data = pd.concat([pd.read_csv(data_path, sep=";") for data_path in data_paths])
all_data["timestamp"] = all_data["timestamp"]//100
all_data["timestamp"] = all_data["timestamp"] + (all_data["day"])*10000
all_data.drop("profit_and_loss", axis=1, inplace=True)

products = all_data["product"].unique()
data_by_product = {}
for product in products:
    data_by_product[product] = all_data[all_data["product"] == product]
    data_by_product[product].set_index("timestamp", inplace=True)
    data_by_product[product] = data_by_product[product].drop(columns=["day"])
    data_by_product[product].sort_index(inplace=True)

columns = data_by_product["GIFT_BASKET"].columns
components = products[products != "GIFT_BASKET"]
gift_basket_components = pd.DataFrame({"timestamp": data_by_product["GIFT_BASKET"].index, "product": "GIFT_BASKET_COMPONENTS"})
gift_basket_components.set_index("timestamp", inplace=True)
for column in columns:
    if "price" in column:
        gift_basket_components[column] = sum([data_by_product[product][column] for product in components])
    elif "volume" in column:
        gift_basket_components[column] = np.min([data_by_product[product][column] for product in components], axis=0)

data_by_product["GIFT_BASKET_COMPONENTS"] = gift_basket_components

# Helper functions

# Plot basket and component costs

In [25]:
data_by_product["GIFT_BASKET_COMPONENTS"]

,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price
timestamp,,,,,,,,,,,,,,
0,GIFT_BASKET_COMPONENTS,26997,72,NaN,NaN,NaN,NaN,27003,72,NaN,NaN,NaN,NaN,27000.0
1,GIFT_BASKET_COMPONENTS,26999,52,NaN,NaN,NaN,NaN,27003,66,NaN,NaN,NaN,NaN,27001.0
2,GIFT_BASKET_COMPONENTS,26999,61,NaN,NaN,NaN,NaN,27003,21,NaN,NaN,NaN,NaN,27001.0
3,GIFT_BASKET_COMPONENTS,27000,44,NaN,NaN,NaN,NaN,27004,69,NaN,NaN,NaN,NaN,27002.0
4,GIFT_BASKET_COMPONENTS,26997,62,NaN,NaN,NaN,NaN,27001,62,NaN,NaN,NaN,NaN,26999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,GIFT_BASKET_COMPONENTS,26142,54,NaN,NaN,NaN,NaN,26145,54,NaN,NaN,NaN,NaN,26143.5
29996,GIFT_BASKET_COMPONENTS,26140,35,NaN,NaN,NaN,NaN,26144,35,NaN,NaN,NaN,NaN,26142.0
29997,GIFT_BASKET_COMPONENTS,26145,48,NaN,NaN,NaN,NaN,26148,65,NaN,NaN,NaN,NaN,26146.5


In [26]:
data_by_product["GIFT_BASKET"]

,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price
timestamp,,,,,,,,,,,,,,
0,GIFT_BASKET,71348,19,71347.0,24.0,NaN,NaN,71362,19,71363.0,24.0,NaN,NaN,71355.0
1,GIFT_BASKET,71344,1,71343.0,12.0,71342.0,20.0,71355,1,71356.0,12.0,71357.0,20.0,71349.5
2,GIFT_BASKET,71344,4,71339.0,1.0,71338.0,14.0,71350,1,71351.0,14.0,71352.0,21.0,71347.0
3,GIFT_BASKET,71345,2,71343.0,11.0,71342.0,29.0,71356,2,71357.0,11.0,71358.0,29.0,71350.5
4,GIFT_BASKET,71336,2,71335.0,13.0,71334.0,25.0,71347,2,71348.0,13.0,71349.0,25.0,71341.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,GIFT_BASKET,69528,20,69527.0,23.0,NaN,NaN,69541,20,69542.0,23.0,NaN,NaN,69534.5
29996,GIFT_BASKET,69523,14,69522.0,23.0,NaN,NaN,69536,14,69537.0,23.0,NaN,NaN,69529.5
29997,GIFT_BASKET,69537,13,69536.0,17.0,NaN,NaN,69549,13,69551.0,17.0,NaN,NaN,69543.0
